In [2]:
# import json
import ast
from tqdm import tqdm_notebook

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.max_colwidth = 150

# Импорт данных

In [4]:
dataset = pd.read_parquet("../data/full_dataset.parquet")
etl = pd.read_parquet("../data/train_data.parquet")

In [5]:
dataset.head()

,variantid1,variantid2,target
0,587809782,615149925,1.0
1,89598677,89581630,1.0
2,539055461,381713564,1.0
3,539055461,358974091,1.0
4,381713564,358974091,1.0


In [6]:
# Пары + признаки
features = (
    dataset
    .merge(
        etl
        .add_suffix('1'),
        on="variantid1"
    )
    .merge(
        etl
        .add_suffix('2'),
        on="variantid2"
    )
)

In [35]:
df = features[['target', 'variantid1', 'variantid2', 'categories1', 'categories2','name1', 'name2','characteristic_attributes_mapping1', 'characteristic_attributes_mapping2']]

# Categories
* сравниваю словари категорий в разрезе таргета, кажется что в парах с целевой меткой доля одинаковых словарей должна быть больше

In [36]:
def compare_dicts(row):
    """
    Функция сравнивает 2 строки-словаря друг с другом.
    """
    return row['categories1'] == row['categories2']


df['categories1'] = df['categories1'].str.lower()
df['categories2'] = df['categories2'].str.lower()
df['comparison_result'] = df.apply(compare_dicts, axis=1) * 1

In [37]:
df.groupby('target', as_index=False).agg({'comparison_result':np.mean})

,target,comparison_result
0,0.0,0.976944
1,1.0,0.967852


* как то странно, казалось что в парах с таргетом совпадений должно быть больше
* посмотрим на такие странные пары

In [38]:
cat_missmatch = df[(df['target'] == 1) & (df['comparison_result'] == 0)]

In [39]:
def miss_match_info(cat_missmatch):
    """
    Функция берет на вход дф со строками словарями категорий.
    Возвращает новый дф с тремя столбцами:
    missmatch_cat - 1й уровень категории где есть различие в паре. Если расхождение было в категориях 2,3,4 то вернется 2.
    miss_match_obj - словарь, где ключ - категория с мисметчем по паре, значение - кортеж с названиями категорий
    missmatch_cat_cnt - в скольких уровнях был мисметч. Если расхождение было в категориях 2,3,4 то вернется 3.
    """

    miss_match_series = pd.Series(index=cat_missmatch.index, dtype='object')
    miss_match_category = pd.Series(index=cat_missmatch.index, dtype='int8')
    miss_match_count_category = pd.Series(index=cat_missmatch.index, dtype='int8')

    for i in cat_missmatch.index:
        cat1_dict = ast.literal_eval(cat_missmatch.loc[i,'categories1'])
        cat2_dict = ast.literal_eval(cat_missmatch.loc[i,'categories2'])
        if len(cat1_dict) == len(cat2_dict):
            miss_match_dict = {}
            cat_list = []
            for key in cat1_dict.keys():

                if cat1_dict[key] != cat2_dict[key]:
                    miss_match_dict[key] = tuple((cat1_dict[key], cat2_dict[key]))
                    miss_match_series.loc[i] = str(miss_match_dict)
                    cat_list.append(int(key))
            miss_match_category.loc[i] = min(cat_list)
            miss_match_count_category.loc[i] = len(cat_list)
            
    return pd.concat([miss_match_category, miss_match_series, miss_match_count_category], ignore_index=False, axis=1)

In [40]:
cat_missmatch = pd.concat([cat_missmatch, miss_match_info(cat_missmatch)], ignore_index=False, axis=1)

In [31]:
cat_missmatch.head()

,target,categories1,categories2,name1,name2,characteristic_attributes_mapping1,characteristic_attributes_mapping2,comparison_result,missmatch_cat,miss_match_obj,missmatch_cat_cnt
29,1.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""жесткие диски, ssd и сетевые накопители"", ""4"": ""внутренний hdd""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""жесткие диски, ssd и сетевые накопители"", ""4"": ""внешний hdd""}","Внутренний жесткий диск Netac Внешний жесткий диск 2,5 1TB K331-1T черный (Внешний жесткий диск 2,5 1TB Netac K331-1T черный)","Внешний жесткий диск Netac NT05K331N-001T-30BK (6926337220963), черный","{""Тип"":[""Внутренний жесткий диск""],""Бренд"":[""Netac""]}","{""Тип"":[""Внешний накопитель""],""Комплектация"":[""Кабель usb""],""Скорость чтения, Мб/с"":[""120""],""Гарантийный срок"":[""36 мес.""],""Цвет товара"":[""черный""...",0,4.0,"{'4': ('внутренний hdd', 'внешний hdd')}",1.0
42,1.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""рюкзаки, чехлы, сумки"", ""4"": ""рюкзак для ноутбука""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""рюкзаки, чехлы, сумки"", ""4"": ""сумка для ноутбука""}","Рюкзак для ноутбука 15"" HP RENEW","Рюкзак для ноутбука 15.6"" HP Renew 14 Backpack (1A212AA), полиэстер, синий","{""Максимальный размер ноутбука, дюймы"":[""15""],""Тип"":[""Рюкзак для ноутбука""],""Количество внутренних отделений"":[""7""],""Количество внешних карманов"":...","{""Внешние размеры, мм"":[""435х295х100""],""Пол"":[""Женский"",""Мужской""],""Количество внешних карманов"":[""1""],""Застежка"":[""Молния""],""Тип"":[""Рюкзак для но...",0,4.0,"{'4': ('рюкзак для ноутбука', 'сумка для ноутбука')}",1.0
43,1.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""рюкзаки, чехлы, сумки"", ""4"": ""рюкзак для ноутбука""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""рюкзаки, чехлы, сумки"", ""4"": ""сумка для ноутбука""}","Рюкзак для ноутбука 15.6"" HP Renew 14 Backpack (1A212AA), полиэстер, синий","Рюкзак для ноутбука 15.6"" HP Renew 14 Backpack (1A212AA), полиэстер, синий","{""Гарантийный срок"":[""12 месяцев""],""Цвет товара"":[""синий""],""Тип"":[""Рюкзак для ноутбука""],""Количество внешних карманов"":[""1""],""Количество внутренни...","{""Внешние размеры, мм"":[""435х295х100""],""Пол"":[""Женский"",""Мужской""],""Количество внешних карманов"":[""1""],""Застежка"":[""Молния""],""Тип"":[""Рюкзак для но...",0,4.0,"{'4': ('рюкзак для ноутбука', 'сумка для ноутбука')}",1.0
45,1.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""рюкзаки, чехлы, сумки"", ""4"": ""сумка для ноутбука""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""рюкзаки, чехлы, сумки"", ""4"": ""рюкзак для ноутбука""}","Рюкзак для ноутбука 15.6"" HP Renew 14 Backpack (1A212AA), полиэстер, синий","Рюкзак для ноутбука 15.6, HP RENEW Navy Backpack, син, 1A212AA","{""Внешние размеры, мм"":[""435х295х100""],""Пол"":[""Женский"",""Мужской""],""Количество внешних карманов"":[""1""],""Застежка"":[""Молния""],""Тип"":[""Рюкзак для но...","{""Цвет товара"":[""синий""],""Максимальный размер ноутбука, дюймы"":[""15""],""Тип"":[""Рюкзак для ноутбука""],""Количество внутренних отделений"":[""2""],""Бренд...",0,4.0,"{'4': ('сумка для ноутбука', 'рюкзак для ноутбука')}",1.0
58,1.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеокамеры"", ""4"": ""видеокамера""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеонаблюдение"", ""4"": ""система видеонаблюдения""}",Видеокамера Hikvision DS-2CD2123G0E-I 2.8мм (DS-2CD2123G0E-I),Видеокамера IP Hikvision DS-2CD2123G0-IS,"{""Бренд"":[""Hikvision""],""Гарантийный срок"":[""официальная гарантия производителя""],""Тип"":[""Видеокамера""],""Цвет товара"":[""белый""]}","{""Скорость съемки в макс. разрешении, кадр/с"":[""25""],""Материал корпуса камеры"":[""Металл""],""Рабочая диафрагма, F"":[""2""],""Цвет товара"":[""белый""],""Ти...",0,3.0,"{'3': ('видеокамеры', 'видеонаблюдение'), '4': ('видеокамера', 'система видеонаблюдения')}",2.0


In [32]:
cat_missmatch['missmatch_cat'].value_counts(normalize=True)

missmatch_cat
4.0    0.646941
3.0    0.326574
2.0    0.026485
Name: proportion, dtype: float64

In [33]:
cat_missmatch['missmatch_cat_cnt'].value_counts(normalize=True)

missmatch_cat_cnt
1.0    0.646941
2.0    0.326574
3.0    0.026485
Name: proportion, dtype: float64

* 2,7 % пар с меткой 1, имеют расхождение со 2-4 категории. Скорее всего это ошибки разметки. Предлагаю сносить.

In [44]:
cat_missmatch_drop = cat_missmatch[["variantid1", "variantid2"]]


In [45]:
dataset = dataset.set_index(["variantid1", "variantid2"])
dataset = dataset.loc[~dataset.index.isin(cat_missmatch_drop.values.tolist())]
dataset = dataset.reset_index()

In [47]:
dataset.to_parquet("../data/full_dataset.parquet", index=False)

##  Parsing categories 

In [3]:
etl = pd.read_parquet("../data/train_data.parquet")
test_etl = pd.read_parquet("../data/test_data.parquet")

In [4]:
dataset = pd.read_parquet("../data/full_dataset.parquet")

In [5]:
from tqdm import tqdm
def parse_category(features: pd.DataFrame) -> pd.DataFrame:
    df_cat = pd.DataFrame({f"cat_{i+1}": [pd.NA] * features.shape[0] for i in range(4) })
    for i in tqdm(range(features.shape[0])):    
        cat1 = ast.literal_eval(features.loc[i,'categories'])    
    
        for j in range(4):  
            df_cat.loc[i,f"cat_{j+1}"] = cat1[str(j+1)]
    return pd.concat([features, df_cat], axis=1)

In [6]:
etl = parse_category(etl)
etl = etl.drop(columns=["categories"])
test_etl = parse_category(test_etl)
test_etl = test_etl.drop(columns=["categories"])

100%|██████████| 35730/35730 [00:05<00:00, 6245.42it/s]


In [7]:
test_etl[test_etl["cat_2"] != "Электроника"]

,variantid,name,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,characteristic_attributes_mapping,cat_1,cat_2,cat_3,cat_4


In [8]:
drop_variant = etl.loc[etl["cat_2"] != "Электроника", "variantid"]
etl = etl[etl["cat_2"] == "Электроника"] # save only "Электроника"
etl = etl.drop(columns=["cat_1", "cat_2"])
test_etl = test_etl.drop(columns=["cat_1", "cat_2"])
etl.to_parquet("../data/train_data.parquet")
test_etl.to_parquet("../data/test_data.parquet")

In [9]:
dataset = dataset[~dataset["variantid1"].isin(drop_variant)]
dataset = dataset[~dataset["variantid2"].isin(drop_variant)]
dataset.to_parquet("../data/full_dataset.parquet")

In [10]:
etl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 456843 entries, 0 to 457062
Data columns (total 9 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   variantid                          456843 non-null  int64 
 1   name                               456843 non-null  object
 2   color_parsed                       378543 non-null  object
 3   pic_embeddings_resnet_v1           303317 non-null  object
 4   main_pic_embeddings_resnet_v1      456843 non-null  object
 5   name_bert_64                       456843 non-null  object
 6   characteristic_attributes_mapping  456816 non-null  object
 7   cat_3                              456843 non-null  object
 8   cat_4                              456843 non-null  object
dtypes: int64(1), object(8)
memory usage: 34.9+ MB


In [41]:
from typing import Any, Dict


def tuplizer(x):
    return tuple(x) if isinstance(x, (np.ndarray, list)) else x

def translate_colors(color: Any):
    color_mapping: Dict[str, str] = {
    "black": "черный", 
    "white": "белый", 
    "pink": "розовый",
    "violet": "фиолетовый", 
    "gold": "золотистый", 
    "золотой": "золотистый",
    "red": "красный", 
    "brown": "коричневый",
    "amethyst": "аметистовый", 
    "khaki": "хаки", 
    "gray": "серый", 
    "grey": "серый", 
    "beige": "бежевый", 
    "orange": "оранжевый", 
    "peach": "персиковый", 
    "bronze": "бронзовый", 
    "purple": "пурпурный",
    "lemon": "лимонный", 
    "emerald": "изумрудный", 
    "silver": "серебряный", 
    "blue": "синий",
    "fuchsia": "фуксия",
    "cyan": "циан",
    "yellow": "желтый",
    "green": "зеленый", 
    "камуфляжный": "хаки",
    "silver": "серебристый",
    "серебряный": "серебристый",
    "мята": "мятный",
    "orchid": "орхидея", 
    "aqua": "аква", 
    "emeral": "изумрудный",
    "крас": "красный",
    "красн": "красный", 
    "изумруд": "изумрудный",
    "chocolate": "шоколадный", 
    "tomato": "томатный", 
    "olive": "оливковый", 
    "бел": "белый", 
    "pear": "грушевый", 
    "lime": "лаймовый", 
    "lavender": "лавандовый", 
    "sapphire": "сапфировый", 
    "белоснежный": "белый", 
    "лайм": "лаймовый", 
    "зел": "зеленый", 
    "cream": "кремовый",
    "snow": "белый",
    "lilac": "лиловый",
    "лаванда": "лавандовый",
    "cobalt": "кобальтовый",
    "copper": "медный",
    "фиол": "фиолетовый",
    "чер": "черный",
    "jade": "нефритовый", 
    "indigo": "индиго", 
    "amber": "янтарный", 
    "желт": "желтый",
    "тыква": "тыквенный",
    "teal": "бирюзовый",
    "turquoise": "бирюзовый",
    "син": "синий",
    "сер": "серый",
    "черн": "черный",
    "vanilla": "ванильный",
    "голуб": "голубой",
    "фиолет": "фиолетовый",
    "flax": "льняной",
    "linen ": "льняной",
    "мультиколор": "многоцветный",
    "разноцветный": "многоцветный",
    "brass": "латунный",
    "aquamarine": "аквамариновый",
    "burgundy": "бордовый", 
    "бордо": "бордовый", 
    "вишня": "вишневый", 
    "azure": "синий", 
    }
    if color is not None:
        color = set([color_mapping[c]
                     if c in color_mapping
                     else c.split("-")[-1]
                     for c in color])
    return color
etl["color_parsed"] = etl["color_parsed"].apply(translate_colors)
test_etl["color_parsed"] = test_etl["color_parsed"].apply(translate_colors)

In [43]:
etl.to_parquet("../data/train_data.parquet")
test_etl.to_parquet("../data/test_data.parquet")

In [30]:
colors = etl["color_parsed"].apply(tuplizer).unique()
test_colors = test_etl["color_parsed"].apply(tuplizer).unique()

In [23]:
unique_colors = set()
for color in colors:
    if color is not None:
        unique_colors = unique_colors.union(set(color))
with open("colors.txt", "w") as f:
    f.writelines("\n".join(unique_colors)) 

In [32]:
test_unique_colors = set()
for color in test_colors:
    if color is not None:
        test_unique_colors = test_unique_colors.union(set(color))
with open("test_colors.txt", "w") as f:
    f.writelines("\n".join(test_unique_colors)) 
